# Insper - Ciência dos Dados 2022.2

## Engenharia - 2ºB

### Integrantes do grupo:

* Beatriz Rodrigues de Freitas

* Carlos Eduardo Porciuncula Yamada

* Verônica Luisa Ribeiro Lima

___
<div id='sumario'></div>

## Sumário

1. [Introdução](#intro)

2. [Minerando Dados e Características do Dataset](#data-mining)

3. [Modelos de Predição](#prediction)

4. [Processo e Estatísticas de Validação](#validation)

5. [Conclusão](#conclusion)

6. [Referências Bibliográficas](#ref)

___
<div id='intro'></div>

## 1. Introdução

### Objetivo

O objetivo deste projeto é criar um código capaz de prever o preço de um notebook (em euros) a partir de algumas features fornescidas pela base de dados "Laptop_price", disponibilizada na plataforma Kaggle por MUHAMMET VARLI em 2020. 
Os dados utilizados para a modelagem da target estão descritos a seguir:


### Descrição da base de dados

* `Company`: **object** → Fabricante do computador;

* `TypeName`: **object** → Tipo do uso;

* `Inches`: **float** → Tamanho (em polegadas);

* `Cpu`: **object** → Qual o processador;

* `Ram`: **object** → Quantidade de memória Ram;

* `Memory`: **object** → Quantidade de memória Rom, assim como o seu tipo;

* `Product`: **object** → Modelo do notebook;

* `ScreenResolution`: **object** → Resolução da tela;

* `OpSys`: **object** → Sistema operacional.


As bibliotecas utilizadas na contrução do robô serão as seguintes:

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn.model_selection import train_test_split
import os
import re 
import matplotlib.pyplot as plt

Chamando a bade de dados:

In [2]:
laptop_price = pd.read_csv('dados/laptop_price.csv',sep=',',  encoding='latin-1')
laptop_price.head(3)

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00


[Voltar para o sumário](#sumario)

___
<div id='data-mining'></div>

## 2. Minerando Dados e Características do Dataset




O primeiro passo foi identificar quais seriam as features que modelam o preço. Para tal, foi utilizado o **groupby** e o **describe** para ver a relação de cada coluna com o valor do notebook, como se observa no arquivo **Describes**. 
Juntando esses resultados com o conhecimento prévio acerca de quais componentes mais influenciam no preço de um computador, foi decidido não levar em consideração a resulução da tela, o peso e o sistema operacinal.

Um problema das features CPU, Memory e GPU é a grande quantidade de opções: 118, 39 e 110, respectivamente, o que pode interferir na eficiência do código.
Não seria viável reagrupar as Gpu's, pois há uma grande variação de preço dentro de uma mesma categoria (uma Nvidia GeForce GTX 500 custa menos da metade de uma Nvidia GeForce GTX 1650, por exemplo) - o mesmo ocorre com "Memory", pois 256GB em SSD é mais caro do que a mesma quantidade em um HD. Assim, a única feature que pode ter as categorias resumidas sem maiores perdas seria o processador. Para tal, observou-se quais os mais recorrentes:

In [14]:
laptop_price['Cpu'].value_counts()

Intel Core i5 7200U 2.5GHz       190
Intel Core i7 7700HQ 2.8GHz      146
Intel Core i7 7500U 2.7GHz       134
Intel Core i7 8550U 1.8GHz        73
Intel Core i5 8250U 1.6GHz        72
                                ... 
Intel Core M M3-6Y30 0.9GHz        1
AMD A9-Series 9420 2.9GHz          1
Intel Core i3 6006U 2.2GHz         1
AMD A6-Series 7310 2GHz            1
Intel Xeon E3-1535M v6 3.1GHz      1
Name: Cpu, Length: 118, dtype: int64

A partir dessa contagem, as Cpu's foram separadas nas seguintes categorias, seguindo a recorrência: Intel Core I7, Intel Core I5, Intel Core I3, Intel Pentium, Intel Celeron, AMD e outros.

In [7]:
dados = laptop_price.loc[:, ['Company', 'TypeName', 'Cpu', "Inches", 'Ram', 'Memory', 'Gpu', 'Price_euros']]

dados.insert(3, 'Processor', '')  # --> onde será armazenado as novas categorias de processador

tamanho = len(dados['Company'])

for i in range(0, tamanho):
    cpu = dados["Cpu"][i]

    if re.search("AMD", cpu):
        dados.loc[i,'Processor']= 'AMD'
    elif re.search("Intel Pentium", cpu):
        dados.loc[i,'Processor']= 'Intel Pentium'
    elif re.search("Intel Core i3", cpu):
        dados.loc[i,'Processor']= 'Intel Core i3'
    elif re.search("Intel Core i5", cpu):
        dados.loc[i,'Processor']= 'Intel Core i5'
    elif re.search("Intel Core i7", cpu):
        dados.loc[i,'Processor']= 'Intel Core i7'
    elif re.search("Intel Celeron", cpu):
        dados.loc[i,'Processor']= 'Intel Celeron'
    else:
        dados.loc[i,'Processor']= 'Outros'

dados.drop(columns = 'Cpu', axis = 1, inplace=True)
dados.head(3)

,Company,TypeName,Processor,Inches,Ram,Memory,Gpu,Price_euros
0,Apple,Ultrabook,Intel Core i5,13.3,8GB,128GB SSD,Intel Iris Plus Graphics 640,1339.69
1,Apple,Ultrabook,Intel Core i5,13.3,8GB,128GB Flash Storage,Intel HD Graphics 6000,898.94
2,HP,Notebook,Intel Core i5,15.6,8GB,256GB SSD,Intel HD Graphics 620,575.00


[Voltar para o sumário](#sumario)

___
<div id='prediction'></div>

## 3. Modelos de Predição

Descrever e justificar pelo menos **DUAS** técnicas de predição.

[Voltar para o sumário](#sumario)

___
<div id='validation'></div>

## 4. Processo e Estatísticas de validação

Calcular medidas que informam a performance de cada modelo ajustado. Para cada um:

* Dividir a base de dados em **Treinamento** e **Teste**. Usar **Treinamento** para estimar cada modelo.

* Estudar medidas que validam os modelos.

    Exemplos:

    * <a href='https://help.salesforce.com/s/articleView?id=sf.bi_edd_model_metrics_cross_validation_measure.htm&type=5'>Cross Validation Tab for Numeric Use Cases</a>

    * <a href='https://paulovasconcellos.com.br/como-saber-se-seu-modelo-de-machine-learning-está-funcionando-mesmo-a5892f6468b'>Como saber se seu modelo de Machine Learning está funcionando mesmo</a>

    * <a href='https://ensinandomaquinasblog.wordpress.com/2017/12/15/modelos-preditivos-de-notas-de-redacao-do-enem-2015/'>Modelos Preditivos de Notas de Redação do ENEM 2015</a>

* Discutir sobre overfitting e underfitting (<a href='https://machinelearningmastery.com/overfitting-and-underfitting-with-machine-learning-algorithms/'>link</a>)

* **Extra**: Faça o processo de Validação Cruzada utilizando também 10 ciclos e calcule a performance média e desvio padrão das duas medidas R2 e RMS tanto para a parte treinamento como para a parte teste. Discuta com riqueza de detalhes.

[Voltar para o sumário](#sumario)

___
<div id='conclusion'></div>

## 5. Conclusão

[Voltar para o sumário](#sumario)

___
<div id='ref'></div>

## 6. Referências Bibliográficas

- Banco de dados:
https://www.kaggle.com/datasets/muhammetvarl/laptop-price

[Voltar para o sumário](#sumario)